In [ ]:
from bokeh.models import DatetimeTickFormatter, PrintfTickFormatter
from IPython.display import Markdown as md
import holoviews as hv
from holoviews import opts
hv.extension('bokeh', logo=False)
import numpy as np

from lsst.sitcom.mareuter.dsm.data_handler import DataHandler, REPORT_TIME_FORMAT

In [ ]:
ipath = "~/Dropbox_LSST/Dome_Seeing_Monitor/DSM_Data/20200122/40FPS"
date_str = "20200122_195044"
csc_index = 1
efd_name = None
use_old_weather = False

In [ ]:
data = DataHandler(ipath, date_str, 0.85)

In [ ]:
await data.process(efd_name, csc_index, use_old_weather)

In [ ]:
md(f"### Dome Seeing Monitor Report {data.report_time()}")

In [ ]:
print("General Information")
data.print_general_information()
print()
print("Camera Information")
data.print_camera_information()
print()
data.print_extra_information()

In [ ]:
psd_x = hv.Image((data.frequencies, data.timestamps, data.psd_x), rtol=1, label='PSD X').opts(width=460)
psd_y = hv.Image((data.frequencies, data.timestamps, data.psd_y), rtol=1, label='PSD Y').opts(yaxis='bare', width=350)

psd = psd_x + psd_y

colorbar_opts = {'colorbar': True, 'colorbar_position': 'bottom',
                 'colorbar_opts': {'formatter': PrintfTickFormatter(format="%0.0e")}}

dt_format = REPORT_TIME_FORMAT
formats = {'days': dt_format, 'months': dt_format, 'hours': dt_format, 'minutes': dt_format}
date_formatter = DatetimeTickFormatter(**formats)

psd.opts(opts.Image(cmap='viridis', logz=True, invert_yaxis=True,
                    **colorbar_opts,
                    height=400,
                    yformatter=date_formatter,
                    xlabel='𝜈 (Hz)', ylabel='Date'))

In [ ]:
src = hv.Table(data.pd_seeing.reset_index())
src_w = hv.Table(data.pd_weather.reset_index())
tick_rotation = 75
x_tuple = ('index', 'Date')
full_width = 350
end_col_size = full_width - 75
wind_speed = "avg2M" if use_old_weather else "speed"

cx = hv.Curve(src, x_tuple, ('centroidX', 'X'), label='X')
cy = hv.Curve(src, x_tuple, ('centroidY', 'Y'), label='Y')
centroids = (cx * cy).opts(ylabel='Pixel Position', title='Centroids', width=full_width)
sx = hv.Curve(src, x_tuple, ('rmsX', 'X'), label='X')
sy = hv.Curve(src, x_tuple, ('rmsY', 'Y'), label='Y')
seeing = (sx * sy).opts(ylabel='scaled RMS (arcsec)', title='Seeing', width=full_width)
wind = hv.Curve(src_w, x_tuple, (wind_speed, 'Speed (m/s)'), label='Wind').opts(width=end_col_size)
flux = hv.Curve(src, x_tuple, ('flux', 'Flux'), label='Flux').opts(ylabel='')
max_adc = hv.Curve(src, x_tuple, ('maxADC', 'Max ADC'), label='Max ADC').opts(ylabel='')
fwhm = hv.Curve(src, x_tuple, ('fwhm', '(pixels)'), label='FWHM').opts(width=end_col_size)

layout = (centroids + seeing + wind + flux + max_adc + fwhm).cols(3)
layout.opts(opts.Layout(toolbar='above'),
            opts.Overlay(legend_position='right'),
            opts.Curve(xformatter=date_formatter, xrotation=tick_rotation, height=400))

In [ ]:
hcx = hv.Histogram(np.histogram(src["centroidX"], bins=20), kdims=["X"])
hcy = hv.Histogram(np.histogram(src["centroidY"], bins=20), kdims=["Y"])
hlayout = hcx + hcy
hlayout.opts()

In [ ]:
dcx = src["centroidX"][1:] - src["centroidX"][:-1]
dcy = src["centroidY"][1:] - src["centroidY"][:-1]
hdcx = hv.Histogram(np.histogram(dcx, bins=21), kdims=["ΔX (pixels)"])
hdcy = hv.Histogram(np.histogram(dcy, bins=21), kdims=["ΔY (pixels)"])
dhlayout = hdcx + hdcy
dhlayout.opts()

In [ ]:
psd_x_d = {data.timestamps[i]:hv.Curve((data.frequencies, data.psd_x[i])) for i in range(data.timestamps.shape[0])}
psd_y_d = {data.timestamps[i]:hv.Curve((data.frequencies, data.psd_y[i])) for i in range(data.timestamps.shape[0])}
hm_psd_x = hv.HoloMap(psd_x_d, kdims="datetime")
hm_psd_y = hv.HoloMap(psd_y_d, kdims="datetime")
map_layout = hm_psd_x + hm_psd_y
map_layout.opts(opts.Curve(xlabel="𝜈 (Hz)", ylabel="Power", logy=True))